In [1]:
import time
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.formula.api as smf
import statsmodels.regression.linear_model as slin

from sklearn import linear_model

In [2]:
predictor_cols = ['Group', 'Sex', 'Age']

### 1. Load sample input dataset

This dataset contains `exog` (Group, Sex and Age) and `endog` variables (fMRI brain map interconnections)

In [3]:
data = pd.read_csv(
    '/Users/ljubomir/Documents/neurohackademy_2021/project/NBPy/data/sample_input_1.csv',
    sep=",",
    header=0
).apply(pd.to_numeric, errors='ignore')
data.columns = [c.replace('.', '_') for c in data.columns]
data.head()

,Group,Sex,Age,FAG_FAD,FAG_F1G,FAD_F1G,FAG_F1D,FAD_F1D,F1G_F1D,FAG_F1OG,...,ORD_GRD,SMAG_GRD,SMAD_GRD,COBG_GRD,COBD_GRD,FMG_GRD,FMD_GRD,FMOG_GRD,FMOD_GRD,GRG_GRD
0,Control,F,8.52,0.353834,-0.079097,-0.142089,-0.098187,0.182184,0.136119,-0.136528,...,-0.227927,0.183839,0.024063,0.375259,0.291237,0.334427,0.422445,0.523587,0.556687,0.819152
1,Control,M,16.16,1.063398,-0.007951,-0.221031,-0.415336,-0.500766,0.711215,-0.166893,...,-0.118639,0.207211,0.211242,0.350441,0.297225,0.014917,0.089388,0.122101,0.084486,0.765234
2,Patient,M,17.75,1.268696,0.308791,0.250510,-0.044810,-0.015975,0.250217,-0.376892,...,0.000284,-0.257270,-0.211604,0.405767,0.455390,0.197229,0.422473,0.871838,0.769304,1.475135
3,Control,M,12.27,0.483665,0.056052,-0.199248,-0.114940,-0.006603,0.196900,-0.103025,...,-0.207249,-0.137101,-0.137722,0.214525,0.227899,0.390530,0.494048,0.469539,0.183129,0.874235
4,Control,F,12.07,0.735635,0.168888,0.069894,-0.466187,-0.212627,0.144734,-0.400744,...,-0.213785,-0.324647,-0.309591,0.434704,0.326873,0.011111,0.253135,0.660290,0.605073,0.835976


### 2. Linear Models 

Generate two functions for calculating linear model by utilizing:
- `statsmodels` library
- `sklearn.linear_model` library

Compare performance in terms of speed (execution time)

In [4]:
N = 1000
endog_cols = data.drop(columns=predictor_cols).columns
random_endogs = [endog_cols[r] for r in np.random.randint(0, endog_cols.shape[0], N)]
endog_cols[:10]

Index(['FAG_FAD', 'FAG_F1G', 'FAD_F1G', 'FAG_F1D', 'FAD_F1D', 'F1G_F1D',
       'FAG_F1OG', 'FAD_F1OG', 'F1G_F1OG', 'F1D_F1OG'],
      dtype='object')

#### 2a. Define statsmodel 

In [5]:
def fit_linear_model_stats(data, endog, exog_relation):
    """
    Generate linear model with statsmodels.ols. 
    """
    
    lm = smf.ols(
        formula='%s ~ %s' % (endog, exog_relation), 
        data=data
    ).fit()
    
    df_result = pd.concat([
        lm.pvalues,
        lm.tvalues,
    ], axis=1)
    df_result.columns = ['pvalues_%s' % endog, 'tvalues_%s' % endog]
    return df_result.drop(index=['Intercept'])

In [6]:
df_stats = fit_linear_model_stats(data, data.columns[3], 'Group + Sex*Age')
df_stats

,pvalues_FAG_FAD,tvalues_FAG_FAD
Group[T.Patient],0.217202,1.252376
Sex[T.M],0.605337,0.520570
Age,0.355653,0.933738
Sex[T.M]:Age,0.604070,-0.522405


In [7]:
duration = []
for endog in random_endogs:
    st = time.time()
    _ = fit_linear_model_stats(data, endog, 'Group + Sex*Age')
    et = time.time()
    dur = et - st
    duration.append(dur * 10**3)
print('Statsmodel execution for %d randomly selected endog columns finished on average: %.2f ± %.2f [ms]' % (N, np.mean(duration), np.std(duration)))

Statsmodel execution for 1000 randomly selected endog columns finished on average: 3.31 ± 0.73 [ms]


#### 2b. Define sklearn

In [8]:
def data_preprocessing(data):
    """

    :return:
    """
    # Step 2a: Separate data into exog and edog
    X = data[predictor_cols]

    # Step 2b: create dummy values for categorical data in
    column_types = X.dtypes
    object_columns = column_types[column_types == 'object'].index
    if object_columns.shape[0] > 0:
        dummy_columns = pd.get_dummies(X[object_columns], drop_first=True)
        X = pd.concat([dummy_columns, X], axis=1).drop(columns=object_columns)
        # FIXME: we should read this from input, but atm we will keep it here as hardcoded
        X['Sex_m_Age'] = X['Sex_M'] * X['Age']
    else:
        X['Sex_m_Age'] = X['Sex'] * X['Age']

    return X

In [9]:
def fit_linear_model_sklearn(data, endog, exog_relation):
    """

    :param X:  input variables
    :param y:  dependent variables
    :return:
    """
    
    X = data_preprocessing(data)
    y = data[endog]
    
    # Step 1: train model
    lr_model = linear_model.LinearRegression().fit(
        X,
        y
    )
    # Step 2: prediction
    predictions = lr_model.predict(X)

    # Step 3: pval and tval calculation

    # 3a: get coefficients and stack them with intercept
    lr_parameters = np.append(lr_model.intercept_, lr_model.coef_)

    # 3b: append 1s to input data (for intercept)
    X_intercept = np.append(np.ones((len(X), 1)), X, axis=1)
    # 3b: get mean squared error between true values and predictions and scale it
    mse = np.sum((y - predictions) ** 2) / (X_intercept.shape[0] - X_intercept.shape[1])

    # 3c: calculate invariant of input dataset (this might be tricky to do for larger matrix)
    X_inv = np.linalg.inv(np.dot(X_intercept.T, X_intercept)).diagonal()

    # 3d: estimate variance and standard deviation to calculate tvals
    var = mse * X_inv
    std = np.sqrt(var)
    tvals = lr_parameters / std

    # 3e: calculate pvalues
    pvals = [2 * (1 - scipy.stats.t.cdf(np.abs(i), (X_intercept.shape[0] - X_intercept.shape[1]))) for i in tvals]

    # 3f: save it in dataframe
    index = ['Intercept'] + X.columns.tolist()
    columns = ['pvals_%s' % endog, 'tvals_%s' % endog]
    df_result = pd.DataFrame(
        np.vstack((pvals, tvals)).T,
        columns=columns,
        index=index,
    ).iloc[1:]

    return df_result

In [10]:
df_sklearn = fit_linear_model_sklearn(data, data.columns[3], 'Group + Sex*Age')
df_sklearn

,pvals_FAG_FAD,tvals_FAG_FAD
Group_Patient,0.217202,1.252376
Sex_M,0.605337,0.520570
Age,0.355653,0.933738
Sex_m_Age,0.604070,-0.522405


Compare if we get exactly the same matrix

In [11]:
np.sum(np.sum((df_sklearn - df_stats) ** 2))

0.0

In [12]:
duration = []
for endog in random_endogs:
    st = time.time()
    _ = fit_linear_model_sklearn(data, endog, 'Group + Sex*Age')
    et = time.time()
    dur = et - st
    duration.append(dur * 10**3)
print('Sklearn execution for %d randomly selected endog columns finished on average: %.2f ± %.2f [ms]' % (N, np.mean(duration), np.std(duration)))

Sklearn execution for 1000 randomly selected endog columns finished on average: 4.13 ± 0.19 [ms]


### 3. Linear Models Broadcast

Our goal is to apply Linear Model on array/matrix of interconnection values, avoiding iterations/for loops. 


`statsmodel` can't provide this functionality, while `sklearn` can broadcast us complete matrix of coefficients. 

From offical `statsmodel` documentation:

```
Ordinary Least Squares

Parameters

    endogarray_like

        A 1-d endogenous response variable. The dependent variable.
    exogarray_like

        A nobs x k array where nobs is the number of observations and k is the number of regressors. An intercept is not included by default and should be added by the user. See statsmodels.tools.add_constant.

```

In [13]:
X = data_preprocessing(data)
y = data.drop(columns=predictor_cols)
model = slin.OLS(y, X).fit()
model.summary()

ValueError: shapes (48,378) and (48,378) not aligned: 378 (dim 1) != 48 (dim 0)

### 4. Conclusion

Even thouhg statsmodel has better support for statistical calculations (ie. faster computation of p and t vals), p and t vals calculation is embedeed into package


| Feature      | sklearn | statsmodels     |
| :---        |    :----:   |          ---: |
| P/T vals embedeed     | NO       | YES  |
| R alike formula   | NO        | YES     |
| Preprocessing categorical columns | YES | NO | 
| Speed [ms] | 3.3 | 4.1 | 
| __Broadcasting__ | __YES__ | NO | 